In [6]:
try:
    from pyspark.sql import SparkSession
except:
    import findspark
    findspark.init()
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col
    from pyspark.sql.types import DecimalType

In [7]:
spark = SparkSession.builder.appName("PocDecimalVSDouble").getOrCreate()

In [9]:
from decimal import Decimal

In [10]:
# Creamos la sesión de spark
spark = SparkSession.builder.appName("PocDecimalVSDouble").getOrCreate()

# data
data = [("Producto A", "150.56"),
        ("Producto B", "689.12"),
        ("Producto C", "56.78")]

# Create a DataFrame
df = spark.createDataFrame(data, ["nombre", "precio_str"])

constant_decimal = Decimal("0.1")

# hacemos el cast a Decimal con dos cifras decimales
df = df.withColumn("precio", col("precio_str").cast(DecimalType(10,2)))

# resultado de aplicar los tipos incorrectos
df = df.withColumn("impuesto_decimal", df["precio"] * constant_decimal) 
df= df.withColumn("impuesto_double", col("precio_str").cast('double')*constant_decimal)

# Show  DataFrame
df.show()

# Stop  Spark session
spark.stop()

+----------+----------+------+----------------+------------------+
|    nombre|precio_str|precio|impuesto_decimal|   impuesto_double|
+----------+----------+------+----------------+------------------+
|Producto A|    150.56|150.56|          15.056|15.056000000000001|
|Producto B|    689.12|689.12|          68.912|            68.912|
|Producto C|     56.78| 56.78|           5.678| 5.678000000000001|
+----------+----------+------+----------------+------------------+



In [57]:
from decimal import Decimal,getcontext
print(5.6*0.1)
getcontext().prec = 2
print(Decimal(5.6)*Decimal(0.1))

0.5599999999999999
0.56


Es sumamente importante indicar los tipos correctos, como podemos ver en ambos ejemplos que son esencialmente los mismos (uno en spark y otro en python) cuando la precisión es esencial como aplicaciones o cálculos financieras debemos usar Decimal, en la columna *impuesto_decimal* si aplicamos un 0.1 de impuesto los valores son correctos, sin embargo si usamos tipo double o flotante aunque el escalar que los multiplique sea de tipo decimal hay una expansión de los decimales.